**Simplex Estandar**

In [5]:
import sys
import numpy as np
from fractions import Fraction

try:
    import pandas as pd

    pandas_av = True
except ImportError:
    pandas_av = False
    pass

product_names = []  #arreglo que contiene los nombres de  las restricciones
col_values = [] #
z_equation = [] #coeficientes de la funcion objetivo
final_rows = []
solutions = []
x = 'X' #nombre de las variables del problema
z2_equation = []
removable_vars = []
no_solution = """
       ---Error ----
Tu problema puede que no tenga solución.

Puede que esto sea por que los coeficientes de las variables y restricciones son incorrectos
Por favor revisa y vuelve a correr el programa :3

           """


def main():
    #global decimals
    global const_num, prod_nums
    print("""
   CALCULO METODO SIMPLEX: Maximización (<=).
  

   """) # para tomar en cuenta, para un problema de maximización se debe tener todas las restricciones con signo <=
 
    prob_type =1
    global const_names # lista que contiene los nombres que el usuario quiere para la restricciones
    const_num = int(input("Cuantas variables tienes: >")) #le pide al usuario el número  de variables que posee el problema
    prod_nums = int(input("Cuantas restricciones tienes: >"))  # le pide  al usuario el numero de restricciones que poee el problema
    const_names = [x + str(i) for i in range(1, const_num + 1)] #se le asigna los nombres a las variables para que, dependiendo de la cantidad de variables quede de la siguiente forma: X1,X2,...
    for i in range(1, prod_nums + 1):
      prod_val = input("ingrese el nombre de la restricción No {}: >".format(i))# se le pide  al usuario que ingrese los nombres de las restricciones
      product_names.append(prod_val)
    print("__________________________________________________")
    if prob_type == 1:
      #Ingreso de valores de la función objetivo
        for i in const_names:
            try:
                val = float(Fraction(input("ingresa el valor de %s en la ecuacion Z: >" % i))) #se le pide al usuario que ingrese el coeficiente de cada variable en la función objetivo
            except ValueError:
                print("por favor ingresa un numero")
                val = float(Fraction(input("ingresa el valor de %s en la ecuacion Z: >" % i)))# en caso de que el valor que el usuario ingrese  no sea un numero o un valor valido, se le solicita que ingrese un valor correcto
            z_equation.append(0 - int(val)) #cada coeficiente ingresado se añade a la lista de  la  ecuacion objetivo de tal forma que quedan organizados siendo la primera posicion el coeficiente de la variable X1 y asi sucesivamente
        z_equation.append(0) #en caso de que ocurra algun error el valor 0 será puesto por defecto 

        while len(z_equation) <= (const_num + prod_nums):
            z_equation.append(0)
        print("__________________________________________________")
        for prod in product_names:
            for const in const_names:
                try:
                    val = float(Fraction(input("ingresa el valor de %s en %s: >" % (const, prod)))) # se le pide al usuario que ingrese los coeficientes de cada variable por cada una de las restricciones
                except ValueError:
                    print("por favor ingresa un numero")
                    val = float(Fraction(input("ingresa el valor de %s in %s: >" % (const, prod))))
                col_values.append(val)
            equate_prod = float(Fraction(input('%s igual a: >' % prod)))  #  se  le  pide al usuario que  ingrese el valor al que es igual cada restriccion 
            col_values.append(equate_prod)

        final_cols = stdz_rows(col_values)#se llama a una función que esta en el codigo la cual sirve para armar las filas de tal forma que queden los coeficientes de cada restriccion mas los coeficientes de las variables de holgura
        i = len(const_names) + 1 # cantidad de restricciones mas la solución
        while len(const_names) < len(final_cols[0]) - 1:
            const_names.append('X' + str(i)) # se añade a la lista de los nombres de las variables de holgura los nombres de las variables a encontrar  (nombres de la parte de arriba de la tabla)
            solutions.append('X' + str(i)) # se añade los nombres de las variables a la lista solutions
            i += 1
        solutions.append(' Z') #se añade el nombre de la función objetivo 'Z' (son los nombres de la parte lateral derecha de las tablas impresas)
        const_names.append('Solución') #se añade a la lista el nombre 'solución' para hacer referencia a la columna de las respuestas
        final_cols.append(z_equation) # se añade los coeficientes de la ecuacion Z a la lista estandarizada con los coeficientes del sistema de ecuaciones
        final_rows = np.array(final_cols).T.tolist() #se hace la transpuesta a la matriz final_cols y asi quedan los coeficientes de las restricciones separados por cada variable 
        print('\n##########################################')
        maximization(final_cols, final_rows) # se llama a la función maximización, la cual realiza todo el proceso con los valores ya previamente ingresados
    else:
        sys.exit("ingresa un valor adecuado ->" + str(prob_type))


def maximization(final_cols, final_rows):
    row_app = []
    ultima_col = final_cols[-1] # coeficientes de la ecuación Z
    min_last_row = min(ultima_col) # se busca el minimo valor de los coeficientes de la ecuación Z
    min_manager = 1
    print(" TABLA N* 1")
    try:
        final_pd = pd.DataFrame(np.array(final_cols), columns=const_names, index=solutions) # se hace un dataframe que tiene como indices los nombres de las variables y la funcion Z y commo columnas los nombres de las variables y las variables de holgura mas el nombre de la columna solución
        print(final_pd)
    except:
        print('  ', const_names)
        i = 0
        for cols in final_cols:
            print(solutions[i], cols)
            i += 1
    count = 2
    elemento_pivote = 2
    while min_last_row < 0 < elemento_pivote != 1 and min_manager == 1 and count < 6:
        print("*********************************************************")
        ultima_col = final_cols[-1] #fila con los coeficientes de la ecuación Z 
        last_row = final_rows[-1] # se escogen la columna de las respuestas
        min_last_row = min(ultima_col)  # se escoge el valor minimo de la ultima columna
        index_of_min = ultima_col.index(min_last_row) # se saca el indice del elemento minimo para escoger el pivote
        pivot_row = final_rows[index_of_min] # se saca la fila del elemento más pequeño
        index_pivot_row = final_rows.index(pivot_row) # se saca el indice de la columna pivote
        row_div_val = []
        i = 0
        for _ in last_row[:-1]:
            try:
                val = float(last_row[i] / pivot_row[i]) # se divide los valores de las respuestas entre los valores de los coeficientes de las variables de la fila 
                if val <= 0: # para evitar que se escojan valores denominadores al momento de escoger la fila pivote
                    val = 10000000000
                else:
                    val = val
                row_div_val.append(val)
            except ZeroDivisionError:
                val = 10000000000
                row_div_val.append(val)
            i += 1
        min_div_val = min(row_div_val)      #se escoje el mas cercano a cero y positivo
        index_min_div_val = row_div_val.index(min_div_val) #se escoge el indice del valor escogido como pivote
        elemento_pivote = pivot_row[index_min_div_val] # se guarda en una variable el elmento pivote escogido
        pivot_col = final_cols[index_min_div_val]  # se saca la fila pivote
        index_pivot_col = final_cols.index(pivot_col) # se saca el indice 
        row_app[:] = []
        #Se realiza el metodo de Gauss
        for col in final_cols:
            if col is not pivot_col and col is not final_cols[-1]: # se hace el respectivo procedimiento para las filas que no son la fila pivote ni la fila correspondiente a la ecuacion de la funcion objetivo
                form = col[index_of_min] / elemento_pivote # se divide cada valor entre el valor del pivote
                final_val = np.array(pivot_col) * form # se multiplica el valor de la columna pivote por ek valor que se explica anteriormente
                new_col = (np.round((np.array(col) - final_val), 2)).tolist() # se restan los dos valores de tal manera que los valores de la culumna pivote se cancelen y queden en 0
                final_cols[final_cols.index(col)] = new_col # se reemplaza en la lista de coeficientes final_cols

            elif col is pivot_col: # si la columna es la columna pivote
                new_col = (np.round((np.array(col) / elemento_pivote), 2)).tolist()  # se crea una nueva columan dividiendo la fila por el elemento pivote
                final_cols[final_cols.index(col)] = new_col #se reemplazan los valores en la lista de final_cols 
            else:
                form = abs(col[index_of_min]) / elemento_pivote # se  divide el valor absoluto de los coeficientes de la funcion objetivo entre el elemento pivote
                final_val = np.array(pivot_col) * form
                new_col = (np.round((np.array(col) + final_val), 2)).tolist()
                final_cols[final_cols.index(col)] = new_col
        final_rows[:] = [] # se vacia la lista final_rows
        re_final_rows = np.array(final_cols).T.tolist() # se vuelven a asignar los nuevos valores de la transpuesta de la matriz final_cols
        final_rows = final_rows + re_final_rows

        if min(row_div_val) != 10000000000: # si hay ...
            min_manager = 1
        else:
            min_manager = 0
        print('elemento pivote: %s' % elemento_pivote)
        print('columna pivote: ', pivot_row)
        print('fila pivote: ', pivot_col)
        print("\n")
        solutions[index_pivot_col] = const_names[index_pivot_row]

        print("TABLA N* %d " % count)
        try:
            final_pd = pd.DataFrame(np.array(final_cols), columns=const_names, index=solutions)
            print(final_pd) # se imprime la tabla de cada iteración 
        except:
            print("TABLA N* %d " % count)
            print('  ', const_names)
            i = 0
            for cols in final_cols:
                print(solutions[i], cols)
                i += 1
        count += 1
        ultima_col = final_cols[-1]
        last_row = final_rows[-1]
        min_last_row = min(ultima_col)
        index_of_min = ultima_col.index(min_last_row)
        pivot_row = final_rows[index_of_min]
        row_div_val = []
        i = 0
        for _ in last_row[:-1]:
            try:
                val = float(last_row[i] / pivot_row[i])
                if val <= 0:
                    val = 10000000000
                else:
                    val = val
                row_div_val.append(val)
            except ZeroDivisionError:
                val = 10000000000
                row_div_val.append(val)
            i += 1
        min_div_val = min(row_div_val)
        index_min_div_val = row_div_val.index(min_div_val)
        elemento_pivote = pivot_row[index_min_div_val]
        if elemento_pivote < 0:
            print(no_solution)
   




def stdz_rows(column_values):
    final_cols = [column_values[x:x + const_num + 1] for x in range(0, len(column_values), const_num + 1)] #arma una lista  de listas en donde cada una  tiene los coeficientes de las variables de cada  una de las restricciones
    for cols in final_cols:
        while len(cols) < (const_num + prod_nums):
            cols.insert(-1, 0) #añade  ceros a cada  una de las filas dependiendo de las variables de holgura que se generen

    i = const_num #numero de constantes
    for sub_col in final_cols:
        sub_col.insert(i, 1) #remplaza el cero de la variable de holgura de la respectiva restriccion a 1
        i += 1
    return final_cols


if __name__ == "__main__":
    main()

# Se usan las lista y arreglos de (numpy) 


   CALCULO METODO SIMPLEX: Maximización (<=).
  

   
Cuantas variables tienes: >3
Cuantas restricciones tienes: >2
ingrese el nombre de la restricción No 1: >S1
ingrese el nombre de la restricción No 2: >S2
__________________________________________________
ingresa el valor de X1 en la ecuacion Z: >40
ingresa el valor de X2 en la ecuacion Z: >30
ingresa el valor de X3 en la ecuacion Z: >50
__________________________________________________
ingresa el valor de X1 en S1: >2
ingresa el valor de X2 en S1: >3
ingresa el valor de X3 en S1: >4
S1 igual a: >64
ingresa el valor de X1 en S2: >3
ingresa el valor de X2 en S2: >2
ingresa el valor de X3 en S2: >1
S2 igual a: >34

##########################################
 TABLA N* 1
      X1    X2    X3   X4   X5  Solución
X4   2.0   3.0   4.0  1.0  0.0      64.0
X5   3.0   2.0   1.0  0.0  1.0      34.0
 Z -40.0 -30.0 -50.0  0.0  0.0       0.0
*********************************************************
elemento pivote: 4.0
columna pivote:  [4.0, 1.